# TASK:- to predict whether a person makes over 50K per year or not from classic adult dataset using XGBoost.



# data reading and importing required packages

In [185]:
import numpy as np
import pandas as pd

train_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header = None) #reading a training dataset.

test_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test' , skiprows = 1, header = None) #Reading a test dataset.

col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
'marital_status', 'occupation','relationship', 'race', 'sex', 'capital_gain',
'capital_loss', 'hours_per_week', 'native_country', 'wage_class'] 
train_set.columns = col_labels #assigning the collumns name
test_set.columns = col_labels  #assigning the collumns name

In [173]:
train_set.head() # look on the data set.

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


# data preprocesing

In [174]:
train_set.isnull().sum() #is there any null value is present? 

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
wage_class        0
dtype: int64

In [176]:
x_train=train_set.iloc[:,:-1] #deriving independent features from the data set.
x_train.corr()                #checking for correlation if present.


,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
age,1.000000,-0.076646,0.036527,0.077674,0.057775,0.068756
fnlwgt,-0.076646,1.000000,-0.043195,0.000432,-0.010252,-0.018768
education_num,0.036527,-0.043195,1.000000,0.122630,0.079923,0.148123
capital_gain,0.077674,0.000432,0.122630,1.000000,-0.031615,0.078409
capital_loss,0.057775,-0.010252,0.079923,-0.031615,1.000000,0.054256
hours_per_week,0.068756,-0.018768,0.148123,0.078409,0.054256,1.000000


In [111]:
b=[]
for i in col: #finding the column name which has a numerical values.
    if x_train[i].dtype!=object:
        b.append(i)
b        
        
        

['age',
 'fnlwgt',
 'education_num',
 'capital_gain',
 'capital_loss',
 'hours_per_week']

In [177]:
xnew=pd.DataFrame(data=x_train[['age','fnlwgt','education_num','capital_gain','capital_loss','hours_per_week']])
xnew

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
0,39,77516,13,2174,0,40
1,50,83311,13,0,0,13
2,38,215646,9,0,0,40
3,53,234721,7,0,0,40
4,28,338409,13,0,0,40
...,...,...,...,...,...,...
32556,27,257302,12,0,0,38
32557,40,154374,9,0,0,40
32558,58,151910,9,0,0,40
32559,22,201490,9,0,0,20


# data scalling

In [178]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()                         #datascalling operation 
scaled_data=scaler.fit_transform(xnew)

In [179]:
xx=pd.DataFrame(data=scaled_data,columns=xnew.columns)  #conversion of scalled data into dataframe so that we can concat with encoded categorical dataframe.

In [180]:
xx

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
0,0.030671,-1.063611,1.134739,0.148453,-0.21666,-0.035429
1,0.837109,-1.008707,1.134739,-0.145920,-0.21666,-2.222153
2,-0.042642,0.245079,-0.420060,-0.145920,-0.21666,-0.035429
3,1.057047,0.425801,-1.197459,-0.145920,-0.21666,-0.035429
4,-0.775768,1.408176,1.134739,-0.145920,-0.21666,-0.035429
...,...,...,...,...,...,...
32556,-0.849080,0.639741,0.746039,-0.145920,-0.21666,-0.197409
32557,0.103983,-0.335433,-0.420060,-0.145920,-0.21666,-0.035429
32558,1.423610,-0.358777,-0.420060,-0.145920,-0.21666,-0.035429
32559,-1.215643,0.110960,-0.420060,-0.145920,-0.21666,-1.655225


In [181]:
x_train.drop(x_train[['age','fnlwgt','education_num','capital_gain','capital_loss','hours_per_week']],inplace=True,axis=1) #dropping all the numeric columns. 
x_train

,workclass,education,marital_status,occupation,relationship,race,sex,native_country
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba
...,...,...,...,...,...,...,...,...
32556,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,United-States
32557,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States
32558,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,United-States
32559,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,United-States


In [182]:
x_new=pd.concat([xx,x_train],axis=1) # joining scalled data and categorical data 

In [183]:
x_new

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,workclass,education,marital_status,occupation,relationship,race,sex,native_country
0,0.030671,-1.063611,1.134739,0.148453,-0.21666,-0.035429,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
1,0.837109,-1.008707,1.134739,-0.145920,-0.21666,-2.222153,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,-0.042642,0.245079,-0.420060,-0.145920,-0.21666,-0.035429,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States
3,1.057047,0.425801,-1.197459,-0.145920,-0.21666,-0.035429,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States
4,-0.775768,1.408176,1.134739,-0.145920,-0.21666,-0.035429,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,-0.849080,0.639741,0.746039,-0.145920,-0.21666,-0.197409,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,United-States
32557,0.103983,-0.335433,-0.420060,-0.145920,-0.21666,-0.035429,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States
32558,1.423610,-0.358777,-0.420060,-0.145920,-0.21666,-0.035429,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,United-States
32559,-1.215643,0.110960,-0.420060,-0.145920,-0.21666,-1.655225,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,United-States


In [184]:
y_train=train_set.iloc[:,-1]  #deriving output feature or a target variable column
y_train

0        0
1        0
2        0
3        0
4        0
        ..
32556    0
32557    1
32558    0
32559    0
32560    1
Name: wage_class, Length: 32561, dtype: int64

In [ ]:
col=x_new.columns
col
x_new['age'].dtype

# one-hot encoding

In [122]:
a=[]
for i in col:                    #finding a columns which are contain categorical value
    if x_new[i].dtypes == object:
        
        a.append(i)
 
        
x_new=pd.get_dummies(data=x_new, columns=a, drop_first=True) # one hot encoding

In [159]:
x_new.head()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,...,native_country_ Portugal,native_country_ Puerto-Rico,native_country_ Scotland,native_country_ South,native_country_ Taiwan,native_country_ Thailand,native_country_ Trinadad&Tobago,native_country_ United-States,native_country_ Vietnam,native_country_ Yugoslavia
0,0.030671,-1.063611,1.134739,0.148453,-0.21666,-0.035429,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0.837109,-1.008707,1.134739,-0.145920,-0.21666,-2.222153,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,-0.042642,0.245079,-0.420060,-0.145920,-0.21666,-0.035429,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,1.057047,0.425801,-1.197459,-0.145920,-0.21666,-0.035429,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,-0.775768,1.408176,1.134739,-0.145920,-0.21666,-0.035429,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [144]:
x_new.drop(x_new[["native_country_ Holand-Netherlands"]],axis=1,inplace=True) #i have dropped this columns because in given test data set there is not a single record is containing a country name  "native_country_ Holand-Netherlands".

In [143]:
x_new.columns

Index(['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss',
       'hours_per_week', 'workclass_ Federal-gov', 'workclass_ Local-gov',
       'workclass_ Never-worked', 'workclass_ Private',
       'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc',
       'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 11th',
       'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th',
       'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm',
       'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate',
       'education_ HS-grad', 'education_ Masters', 'education_ Preschool',
       'education_ Prof-school', 'education_ Some-college',
       'marital_status_ Married-AF-spouse',
       'marital_status_ Married-civ-spouse',
       'marital_status_ Married-spouse-absent',
       'marital_status_ Never-married', 'marital_status_ Separated',
       'marital_status_ Widowed', 'occupation_ Adm-clerical',
       'occupation_ Armed-Force

In [ ]:
l_unique = y_train.unique() # fteching out the unique values from target column
l_unique
v_unique = np.arange(len(l_unique)) # obtaining values in the range of the length of I_unique
v_unique
y_train.replace(to_replace=l_unique, value=v_unique, inplace=True) # replacing categorical data with numerical values


In [172]:
y_train.head(5)

0    0
1    0
2    0
3    0
4    0
Name: wage_class, dtype: int64


# Model Building

In [145]:
import xgboost as xgb     #importing algorithm
model=xgb.XGBClassifier(objective='binary:logistic') #object creation

In [146]:
model.fit(x_new, y_train)   #Model Training

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

# apply all the the feature engineering technichs on test data which we have applied for train data . 

In [128]:
x_test=test_set.iloc[:,:-1]

x_test

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States
16277,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States
16278,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States
16279,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States


In [129]:
e=[]
for i in col:
    if x_test[i].dtype!=object:
        e.append(i)
e        
        
    

['age',
 'fnlwgt',
 'education_num',
 'capital_gain',
 'capital_loss',
 'hours_per_week']

In [130]:
xnewt=pd.DataFrame(data=x_test[['age','fnlwgt','education_num','capital_gain','capital_loss','hours_per_week']])
xnewt

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
0,25,226802,7,0,0,40
1,38,89814,9,0,0,50
2,28,336951,12,0,0,40
3,44,160323,10,7688,0,40
4,18,103497,10,0,0,30
...,...,...,...,...,...,...
16276,39,215419,13,0,0,36
16277,64,321403,9,0,0,40
16278,38,374983,13,0,0,50
16279,44,83891,13,5455,0,40


In [131]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_datat=scaler.fit_transform(xnewt)

In [132]:
xxt=pd.DataFrame(data=scaled_datat,columns=xnewt.columns)
xxt

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
0,-0.994129,0.353474,-1.196864,-0.142662,-0.218062,-0.031432
1,-0.055417,-0.942391,-0.417886,-0.142662,-0.218062,0.769918
2,-0.777503,1.395450,0.750582,-0.142662,-0.218062,-0.031432
3,0.377835,-0.275397,-0.028397,0.871091,-0.218062,-0.031432
4,-1.499590,-0.812954,-0.028397,-0.142662,-0.218062,-0.832781
...,...,...,...,...,...,...
16276,0.016791,0.245794,1.140071,-0.142662,-0.218062,-0.351972
16277,1.822007,1.248371,-0.417886,-0.142662,-0.218062,-0.031432
16278,-0.055417,1.755221,1.140071,-0.142662,-0.218062,0.769918
16279,0.377835,-0.998420,1.140071,0.576644,-0.218062,-0.031432


In [133]:
y_testt=test_set.iloc[:,-1]
y_testt

0        0
1        0
2        1
3        1
4        0
        ..
16276    0
16277    0
16278    0
16279    0
16280    1
Name: wage_class, Length: 16281, dtype: int64

In [134]:
x_test.drop(x_test[['age','fnlwgt','education_num','capital_gain','capital_loss','hours_per_week']],inplace=True,axis=1)
x_test

,workclass,education,marital_status,occupation,relationship,race,sex,native_country
0,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,United-States
1,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,United-States
2,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,United-States
3,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,United-States
4,?,Some-college,Never-married,?,Own-child,White,Female,United-States
...,...,...,...,...,...,...,...,...
16276,Private,Bachelors,Divorced,Prof-specialty,Not-in-family,White,Female,United-States
16277,?,HS-grad,Widowed,?,Other-relative,Black,Male,United-States
16278,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States
16279,Private,Bachelors,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,United-States


In [135]:
x_newt=pd.concat([xxt,x_test],axis=1)

In [84]:
x_newt

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,workclass,education,marital_status,occupation,relationship,race,sex,native_country
0,-0.994129,0.353474,-1.196864,-0.142662,-0.218062,-0.031432,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,United-States
1,-0.055417,-0.942391,-0.417886,-0.142662,-0.218062,0.769918,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,United-States
2,-0.777503,1.395450,0.750582,-0.142662,-0.218062,-0.031432,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,United-States
3,0.377835,-0.275397,-0.028397,0.871091,-0.218062,-0.031432,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,United-States
4,-1.499590,-0.812954,-0.028397,-0.142662,-0.218062,-0.832781,?,Some-college,Never-married,?,Own-child,White,Female,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,0.016791,0.245794,1.140071,-0.142662,-0.218062,-0.351972,Private,Bachelors,Divorced,Prof-specialty,Not-in-family,White,Female,United-States
16277,1.822007,1.248371,-0.417886,-0.142662,-0.218062,-0.031432,?,HS-grad,Widowed,?,Other-relative,Black,Male,United-States
16278,-0.055417,1.755221,1.140071,-0.142662,-0.218062,0.769918,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States
16279,0.377835,-0.998420,1.140071,0.576644,-0.218062,-0.031432,Private,Bachelors,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,United-States


In [138]:
k=[]
for i in x_newt.columns:
    if x_newt[i].dtypes == object:
        
        k.append(i)
 
        
x_newt=pd.get_dummies(data=x_newt, columns=k, drop_first=True)

In [139]:
x_newt.head(50)

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,...,native_country_ Portugal,native_country_ Puerto-Rico,native_country_ Scotland,native_country_ South,native_country_ Taiwan,native_country_ Thailand,native_country_ Trinadad&Tobago,native_country_ United-States,native_country_ Vietnam,native_country_ Yugoslavia
0,-0.994129,0.353474,-1.196864,-0.142662,-0.218062,-0.031432,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,-0.055417,-0.942391,-0.417886,-0.142662,-0.218062,0.769918,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,-0.777503,1.395450,0.750582,-0.142662,-0.218062,-0.031432,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0.377835,-0.275397,-0.028397,0.871091,-0.218062,-0.031432,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,-1.499590,-0.812954,-0.028397,-0.142662,-0.218062,-0.832781,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,-0.344252,0.087571,-1.586353,-0.142662,-0.218062,-0.832781,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
6,-0.705295,0.355593,-0.417886,-0.142662,-0.218062,-0.031432,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7,1.749798,-0.802274,1.919049,0.266505,-0.218062,-0.672511,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,-1.066338,1.704933,-0.028397,-0.142662,-0.218062,-0.031432,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
9,1.172129,-0.798774,-2.365331,-0.142662,-0.218062,-2.435480,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [140]:
from sklearn.metrics import accuracy_score

In [147]:
y_pred = model.predict(x_newt)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_testt,predictions)
accuracy

0.865978748234138

In [106]:
l_unique = y_testt.unique() # fteching out the unique values from game_season/
l_unique
v_unique = np.arange(len(l_unique)) # obtaining values in the range of the length of I_unique
v_unique
y_testt.replace(to_replace=l_unique, value=v_unique, inplace=True) # 

In [150]:
y_testt.head()

0    0
1    0
2    1
3    1
4    0
Name: wage_class, dtype: int64

In [151]:
x_newt

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,...,native_country_ Portugal,native_country_ Puerto-Rico,native_country_ Scotland,native_country_ South,native_country_ Taiwan,native_country_ Thailand,native_country_ Trinadad&Tobago,native_country_ United-States,native_country_ Vietnam,native_country_ Yugoslavia
0,-0.994129,0.353474,-1.196864,-0.142662,-0.218062,-0.031432,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,-0.055417,-0.942391,-0.417886,-0.142662,-0.218062,0.769918,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,-0.777503,1.395450,0.750582,-0.142662,-0.218062,-0.031432,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0.377835,-0.275397,-0.028397,0.871091,-0.218062,-0.031432,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,-1.499590,-0.812954,-0.028397,-0.142662,-0.218062,-0.832781,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,0.016791,0.245794,1.140071,-0.142662,-0.218062,-0.351972,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
16277,1.822007,1.248371,-0.417886,-0.142662,-0.218062,-0.031432,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
16278,-0.055417,1.755221,1.140071,-0.142662,-0.218062,0.769918,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
16279,0.377835,-0.998420,1.140071,0.576644,-0.218062,-0.031432,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


# Hyperparameter tunning

In [155]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier hyperparameter tuning

In [156]:
param_grid={
   
    ' learning_rate':[1,0.5,0.1,0.01,0.001],
    'max_depth': [3,5,10,20],
    'n_estimators':[10,50,100,200]
    
}

In [157]:
grid= GridSearchCV(XGBClassifier(objective='binary:logistic'),param_grid, verbose=3) #making a instant for gridsearchcv package

In [158]:
grid.fit(x_new,y_train) # h.p.t applied

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[CV]  learning_rate=1, max_depth=3, n_estimators=10 ..................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]   learning_rate=1, max_depth=3, n_estimators=10, score=0.838, total=   1.0s
[CV]  learning_rate=1, max_depth=3, n_estimators=10 ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV]   learning_rate=1, max_depth=3, n_estimators=10, score=0.840, total=   1.0s
[CV]  learning_rate=1, max_depth=3, n_estimators=10 ..................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.9s remaining:    0.0s


[CV]   learning_rate=1, max_depth=3, n_estimators=10, score=0.846, total=   1.0s
[CV]  learning_rate=1, max_depth=3, n_estimators=10 ..................
[CV]   learning_rate=1, max_depth=3, n_estimators=10, score=0.850, total=   1.0s
[CV]  learning_rate=1, max_depth=3, n_estimators=10 ..................
[CV]   learning_rate=1, max_depth=3, n_estimators=10, score=0.846, total=   1.0s
[CV]  learning_rate=1, max_depth=3, n_estimators=50 ..................
[CV]   learning_rate=1, max_depth=3, n_estimators=50, score=0.855, total=   4.2s
[CV]  learning_rate=1, max_depth=3, n_estimators=50 ..................
[CV]   learning_rate=1, max_depth=3, n_estimators=50, score=0.855, total=   4.2s
[CV]  learning_rate=1, max_depth=3, n_estimators=50 ..................
[CV]   learning_rate=1, max_depth=3, n_estimators=50, score=0.859, total=   4.2s
[CV]  learning_rate=1, max_depth=3, n_estimators=50 ..................
[CV]   learning_rate=1, max_depth=3, n_estimators=50, score=0.860, total=   4.2s
[CV]  l

[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed: 139.6min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={' learning_rate': [1, 0.5, 0.1, 0.01, 0.001],
                         'max_depth': [3, 5, 10, 20],
                         'n_estimators': [10, 50,

In [160]:
grid.best_params_   #for finding a best perameters

{' learning_rate': 1, 'max_depth': 5, 'n_estimators': 200}

In [161]:
new_model=XGBClassifier(learning_rate= 1, max_depth= 5, n_estimators= 200)  #again training model base on hyperparameter tunning
new_model.fit(x_new, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=1,
              max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
              n_estimators=200, n_jobs=1, nthread=None,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1, verbosity=1)

In [163]:
y_pred_new = new_model.predict(x_newt)                   #prediction of test data
predictions_new = [round(value) for value in y_pred_new]
accuracy_new = accuracy_score(y_testt,predictions_new)
accuracy_new

0.8407345986118788

In [169]:
y_pred_new

array([0, 0, 0, ..., 1, 0, 1], dtype=int64)

### XGBClassifier is giving me accuracy=86% without hyperparameter tunning and with hyperparameter tunning accuracy=84%